In [2]:
import subprocess
import json
import os
from web3 import Web3
from dotenv import load_dotenv
from web3.middleware import geth_poa_middleware
from eth_account import Account
from bit import *

from pathlib import Path
from getpass import getpass

In [3]:
load_dotenv()
from constants import *
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))
mnemonic = os.getenv('MNEMONIC')

In [6]:
def derive_wallets(mnenomicvariable, coinname, num):
    x = './wallet/derive -g --mnemonic="' + mnenomicvariable + '" --format=json --coin=' + coinname + ' --numderive='+ str(num) + ' --cols=path,index,address,privkey,pubkey'
    p = subprocess.Popen(x, stdout=subprocess.PIPE, shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    accounts = json.loads(output)
    return accounts

In [7]:
derive_wallets(mnemonic, 'btc-test', 4)[3]['privkey']

'cNUYRV2EeXXvw9zQCwJ1EwidF334onHozbtzjZ1uXbhAZdwa9Yyu'

In [7]:
coins = {'btc': derive_wallets(mnemonic, 'btc', 4),
        'btc-test': derive_wallets(mnemonic, 'btc-test', 4),
        'eth': derive_wallets(mnemonic, 'eth', 4)}

In [8]:
print(json.dumps(coins, indent=2))

{
  "btc": [
    {
      "path": "m/44'/0'/0'/0/0",
      "index": 0,
      "address": "1H18pDC4WTRZwm1hCJGNtpSug77PksmDCn",
      "privkey": "KzRJLkBeGt1fA5tFjJZDeRFSevBuARRKvD5SpZYjLn9omDs1Ef5C",
      "pubkey": "0369f4bbad769e95aa34c00ec0fafed9fd5ab86712c862d346c01b404ee7b004be"
    },
    {
      "path": "m/44'/0'/0'/0/1",
      "index": 1,
      "address": "1LcHvrJXEhbUvUcZAGZQASUrQamjpKahhD",
      "privkey": "L3dTELkRCPvaaEGBwZvG2ko1wJ43RFbsvcmwJ4DNAmUu3kG1ckZV",
      "pubkey": "02a2381866fccc2ee18fa69187f3317882b595935f55c1efe2048df2373b79011e"
    },
    {
      "path": "m/44'/0'/0'/0/2",
      "index": 2,
      "address": "1CPYfsFZ4KiRUSB6GQcUZyKRiS49oRmZ3k",
      "privkey": "L5AEAqU99PtoMDmtW6y7USJZX7m7PHJ6PTkNJJaA1Cwk1QRNkVc1",
      "pubkey": "025ba6d2822d6d9febbceca6c6d097e6d16c5c1aa0b07d3aeb59d4ac5a5fe52dbf"
    },
    {
      "path": "m/44'/0'/0'/0/3",
      "index": 3,
      "address": "1EKzLVUJuGH4Vj8JSD2njAAXHR2aYX2VjQ",
      "privkey": "Ky5w13BmawNMzRxRYLYTwtT9TU

In [9]:
coins['btc'][0]['privkey']

'KzRJLkBeGt1fA5tFjJZDeRFSevBuARRKvD5SpZYjLn9omDs1Ef5C'

In [10]:
def priv_key_to_account(coin, priv_key):
    if coin == 'eth':
        return Account.privateKeyToAccount(priv_key)
    elif coin == 'btc-test': # or 'btc':
        return PrivateKeyTestnet(priv_key)
#    For ETH, return Account.privateKeyToAccount(priv_key)
#    For BTCTEST, return PrivateKeyTestnet(priv_key)

In [11]:
priv_key_to_account('eth', coins['eth'][0]['privkey'] )

In [12]:
priv_key_to_account('btc-test', coins['btc-test'][0]['privkey'])

<PrivateKeyTestnet: mpB463ctKh1qjTK3WdSxB8W12hwCsRnFni>

In [13]:
def create_tx(coin, account, recipient, amount):
    if coin == 'eth':
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": amount}
        )
        return {
            "from": account.address,
            "to": recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
        }
    elif coin == 'btc-test': #or 'btc':
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, 'btc')])

In [14]:
def send_tx(coin, account, recipient, amount):
    tx = create_tx(coin, account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    if coin == 'eth':
        return w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    elif coin == 'btc-test': # or 'btc':
        return NetworkAPI.broadcast_tx_testnet(signed_tx)

In [16]:
create_tx('eth', priv_key_to_account('eth', coins['eth'][0]['privkey']), '0x22C5b0FDeC24947524dd951ACF42564249A3409b', 0)

{'from': '0xdc8fA0017df2e333be878Ca99d52322eB0abaCbd',
 'to': '0x22C5b0FDeC24947524dd951ACF42564249A3409b',
 'value': 0,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 0}

In [17]:
send_tx('eth', priv_key_to_account('eth', coins['eth'][0]['privkey']), '0x22C5b0FDeC24947524dd951ACF42564249A3409b', 0)

HexBytes('0x309d346aa238956a5ff1fdd25e293c3beaa03a8ca84f232a31919dea59da7a82')